In [1]:
import sys
sys.path.append("../")

from src import Estrategia, agrega_estrategias
import sqlite3
import pandas as pd

#Cargar base de días hábiles
dbpath = r'processed_data/db_habiles.db'
conn_habiles = sqlite3.connect(dbpath)
df_dias_habiles = pd.read_sql('SELECT * FROM HABILES',
                         conn_habiles, parse_dates=['Fecha'])

# PARAMETROS PARA DEFINIR LA ESTRATEGIA
#fecha_ini = datetime.datetime.strptime('2014-01-01', '%Y-%m-%d').date()
fecha_ini = pd.to_datetime('2014-01-01').date()
fecha_end = pd.to_datetime('2020-11-23').date()

afp = 'HABITAT'
monto_inicial = 100
lag_solicitud = 0

C:\Users\calvarad\Documents\ACEE\fyf_github\fyf


In [2]:
estrategia_a = Estrategia(fecha_ini, fecha_end, 'A')
estrategia_e = Estrategia(fecha_ini, fecha_end, 'E')
estrategia_ff = Estrategia(fecha_ini, fecha_end, 'FF',
                           'raw_data/anuncios_ff.xlsx')

lista_estrategias = [estrategia_a, estrategia_e, estrategia_ff]

df = agrega_estrategias(lista_estrategias, df_dias_habiles,
                        afp, monto_inicial, lag_solicitud)
display(df.head(5))
display(df.tail(5))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\calvarad\\Documents\\ACEE\\fyf_github\\fyf/raw_data/anuncios_ff.xlsx'

In [ ]:
df.set_index('Fecha').plot()

### 2. Comparemos la estrategia FF versus fondo A y fondo E, con LAG = 2

In [ ]:
df = agrega_estrategias(lista_estrategias, df_dias_habiles, 
                        afp, monto_inicial, lag_solicitud=2)
df.set_index('Fecha').plot()

### 3. Comparemos fondo A versus estrategia FF con 0 y 2 dias de LAG 

In [ ]:
lista_estrategias = [estrategia_a, estrategia_ff, estrategia_ff]

df = agrega_estrategias(lista_estrategias, df_dias_habiles,
                        afp, monto_inicial, [2, 0, 2])
df.set_index('Fecha').plot()

### 4. Comparemos fondo A versus estrategia FF verus estrategia "MAX"

**HIPOTESIS:** Si alguien tiene la capacidad para "predecir" al mercado, su curva de rendimientos debería estar entre la curva pasiva y la curva de mejores rendimientos (Estrategia "MAX").

In [ ]:
estrategia_max = Estrategia(fecha_ini, fecha_end, 'MAX',
                            '../2_avisos_cambio_fondos/optima_HABITAT.xlsx')

lista_estrategias = [estrategia_a, estrategia_max, estrategia_ff]

df = agrega_estrategias(lista_estrategias, df_dias_habiles, afp, monto_inicial, 0)
df.set_index('Fecha').plot()

### 5. ¿Cómo le va a una estrategia en diferentes horizontes de tiempo?

+ Por ejemplo, supongamos que me quiero jubilar en 3 años.
+ Hay diferentes resultados dependiendo del punto de partida. [t, t+ 3 años], [t + x, t + x + 3 años]. Llamemos `offset` a ese `x`
+ Dado un horizonte de tiempo, podemos construir una distribución de resultados para cada `offset` x.

In [ ]:
estrategias_afp = [Estrategia(fecha_ini, fecha_end, fondo) for fondo in ['A', 'B', 'C', 'D', 'E']]

lista_estrategias = estrategias_afp + [estrategia_ff, estrategia_ff]

df = agrega_estrategias(lista_estrategias, df_dias_habiles,
                        afp, monto_inicial, [2, 2, 2, 2, 2, 0, 2])
df.set_index('Fecha').plot()

In [ ]:
horizonte_de_tiempo = pd.Timedelta(3*365, unit="d")
beg_period = df.Fecha.min()
end_period = df.Fecha.max() - horizonte_de_tiempo

all_starting_days_of_experiment = pd.date_range(beg_period, end_period, freq='d')

In [ ]:
relevant_columns = [col for col in df.columns if col != 'Fecha']

rows = []
for start_date in all_starting_days_of_experiment:
    start_data = df.loc[df.Fecha == start_date, relevant_columns]
    end_data = df.loc[df.Fecha == start_date + horizonte_de_tiempo, relevant_columns]
    rows.append(pd.DataFrame(100* (end_data.values / start_data.values - 1), columns=relevant_columns,
                      index=[start_date]))

In [ ]:
results = pd.concat(rows)

display(results.head(3))
display(results.tail(3))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)

sns.distplot(results.Val_A_lag_2, ax=ax, label=f"{afp}_A_lag_2")
sns.distplot(results.Val_C_lag_2, ax=ax, label=f"{afp}_C_lag_2")
sns.distplot(results.Val_E_lag_2, ax=ax, label=f"{afp}_E_lag_2")
sns.distplot(results.Val_FF_lag_2, ax=ax, label=f"FF_lag_2")
ax.legend()

In [ ]:
df.describe()